Source : https://www.kaggle.com/datasets/diamondsnake/eurovision-song-contest-data?resource=download



In [3]:
#import the libraries
import requests
import os
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [4]:
songs_df = pd.read_excel('./data/song_data_cleaned.xls')

#changing loudness category into just integer values
songs_df['loudness_db'] = songs_df['loudness'].str.replace('dB', ' ').str.strip()

#dropping all columns that only have 'unknown' as their single value & original loudness
songs_df = songs_df.drop(['race', 'age', 'selection', 'release_date', 'key_change_10', 'loudness'], axis='columns')


In [5]:
songs_df.columns

Index(['year', 'semi_final', 'semi_draw_position', 'final_draw_position',
       'country', 'artist_name', 'song_name', 'language', 'style',
       'direct_qualifier_10', 'gender', 'main_singers', 'key', 'BPM', 'energy',
       'danceability', 'happiness', 'acousticness', 'instrumentalness',
       'liveness', 'speechiness', 'backing_dancers', 'backing_singers',
       'backing_instruments', 'instrument_10', 'qualified_10',
       'final_televote_points', 'final_jury_points', 'final_televote_votes',
       'final_jury_votes', 'final_place', 'final_total_points', 'semi_place',
       'semi_televote_points', 'semi_jury_points', 'semi_total_points',
       'favourite_10', 'host_10', 'loudness_db'],
      dtype='object')

In [63]:
# Pushing these tables to the AWS

from sqlalchemy import create_engine, types
from sqlalchemy import text # to be able to pass string


# Let's load values from the .env file
from dotenv import dotenv_values

config = dotenv_values()

# define variables for the login
pg_user = config['POSTGRES_USER']  # align the key label with your .env file !
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']


url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

engine = create_engine(url, echo=False)

In [64]:
songs_df.to_sql(name = 'songs', 
                       con = engine, 
                       schema = pg_schema, # pandas is allowing to specify, in which schema the table shall be created
                       if_exists='replace', 
                       index=False
                      )

565